In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
pd.read_csv("Resources/netflix_titles.csv")

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
...,...,...,...,...,...,...,...,...,...,...,...,...
7782,s7783,Movie,Zozo,Josef Fares,"Imad Creidi, Antoinette Turk, Elias Gergi, Car...","Sweden, Czech Republic, United Kingdom, Denmar...","October 19, 2020",2005,TV-MA,99 min,"Dramas, International Movies",When Lebanon's Civil War deprives Zozo of his ...
7783,s7784,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...
7784,s7785,Movie,Zulu Man in Japan,NaN,Nasty C,NaN,"September 25, 2020",2019,TV-MA,44 min,"Documentaries, International Movies, Music & M...","In this documentary, South African rapper Nast..."
7785,s7786,TV Show,Zumbo's Just Desserts,NaN,"Adriano Zumbo, Rachel Khoo",Australia,"October 31, 2020",2019,TV-PG,1 Season,"International TV Shows, Reality TV",Dessert wizard Adriano Zumbo looks for the nex...


In [ ]:
# movie_titles
# -
# ID VARCHAR PK
# show_name VARCHAR
# rating_ID INTEGER FK >- rating.rating_ID
# rating
# -
# rating_ID INTEGER PK
# rating_description VARCHAR
# listed_in
# -
# listed_in_ID INTEGER
# listed_in_description VARCHAR
# info
# -
# ID VARHCAR FK >- movie_titles.ID
# listed_in_ID INTEGER FK >- listed_in.listed_in_ID
# type_ID VARCHAR FK >- type.type_ID
# date_added DATE
# rel_yr VARCHAR
# country VARHCAR
# duration VARCHAR
# description VARCHAR
# type
# -
# type_ID INTEGER PK
# type_desc VARCHAR
# cast
# -
# cast_ID INTEGER PK FK >- crew.cast_ID
# cast VARHCAR
# director
# -
# director_ID INTEGER PK
# director VARCHAR
# crew
# -
# ID VARCHAR FK >- movie_titles.ID
# cast_ID INTGER FK
# director_ID INTEGER FK >- director.director_ID
